# EDA

In [ ]:
from datasets import load_dataset

corpus = load_dataset("castorini/mr-tydi-corpus", "indonesian", trust_remote_code=True)
mr_tydi = load_dataset("castorini/mr-tydi", "indonesian")
tydiqa_gold = load_dataset("khalidalt/tydiqa-goldp", 'indonesian', trust_remote_code=True)

print(corpus)
print(mr_tydi)
print(tydiqa_gold)

Mengecek dataset mr_tydi, terutama terkait apakah:
1. semua positive_passages pada tiap row di ketiga split itu satu
2. kolom negative_passages itu ada isinya atau ngga

In [ ]:
# Mengecek untuk split 'dev' dan 'test'
for split in mr_tydi:
    # Cek apakah semua positive_passages hanya berisi satu elemen
    all_positive_single = all(len(row['positive_passages']) == 1 for row in mr_tydi[split])
    
    # Cek apakah semua negative_passages kosong
    all_negative_empty = all(len(row['negative_passages']) == 0 for row in mr_tydi[split])

    print(f"Split: {split}")
    print(f"- Semua positive_passages berisi satu elemen: {all_positive_single}")
    if all_negative_empty:
        print('- Kolom negative passages kosong')
    else:
        print('- Kolom negative_passages ada isinya')
    print("-" * 50)

Karena kolom positive_passages pada split test tidak semuanya berjumlah satu, maka kita explore lebih dalam

In [ ]:
from collections import Counter

# Hitung jumlah elemen dalam positive_passages untuk setiap row di split 'test'
positive_lengths = [len(row['positive_passages']) for row in mr_tydi['test']]

# Hitung distribusi jumlah elemen dalam positive_passages
length_counts = Counter(positive_lengths)

# Tampilkan hasil
print("Distribusi jumlah elemen dalam positive_passages pada split 'test':")
for length, count in sorted(length_counts.items()):
    print(f"- {count} row memiliki {length} elemen")

Melihat contoh baris yang jumlah positive_passages-nya lebih dari 1

In [ ]:
from collections import defaultdict

# Buat dictionary untuk menyimpan contoh berdasarkan jumlah elemen positive_passages
examples_by_length = defaultdict(list)

# Loop melalui dataset test untuk mengumpulkan contoh
for row in mr_tydi['test']:
    length = len(row['positive_passages'])
    if len(examples_by_length[length]) < 2:  # Simpan maksimal 5 contoh per kategori
        examples_by_length[length].append(row)

# Tampilkan contoh
for length, examples in sorted(examples_by_length.items()):
    if length == 1:
        continue
    print(f"\n===== Contoh untuk positive_passages dengan {length} elemen =====")
    for i, example in enumerate(examples):
        print(f"\nContoh {i+1}:")
        print(f"Query: {example['query']}")
        print(f"Positive Passages: {example['positive_passages']}")

Mengecek distribusi panjang jawaban (dihitung dalam jumlah kata) untuk dataset tydiqa_gold\[train\]

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

# Fungsi untuk menghitung distribusi panjang jawaban
def plot_length_distribution(dataset, ax, title):
    # Hitung distribusi panjang jawaban
    length_counts = Counter(len(row['answers']['text'][0].split()) for row in dataset if row['answers']['text'])
    
    # Urutkan berdasarkan panjang kata
    sorted_lengths = sorted(length_counts.items())
    
    # Ekstrak data untuk plotting
    lengths, counts = zip(*sorted_lengths)
    
    # Plot
    ax.bar(lengths, counts, color='royalblue', alpha=0.7)
    ax.set_xlabel("Jumlah Kata dalam Answer Text")
    ax.set_ylabel("Jumlah Instance")
    ax.set_title(title)
    ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    # Sesuaikan xticks
    ax.set_xticks(range(min(lengths), max(lengths) + 1))  # Menampilkan semua jumlah kata pada sumbu x
    
    # Putar 315 derajat untuk ticks di sumbu X
    for tick in ax.get_xticklabels():
        tick.set_rotation(315)

# Membuat figure dengan 2 plot (satu di atas dan satu di bawah)
fig, axs = plt.subplots(2, 1, figsize=(20, 10), sharex=False)  # 2 baris, 1 kolom

# Plot untuk split 'train'
plot_length_distribution(tydiqa_gold['train'], axs[0], "Distribusi Panjang Jawaban (Train)")

# Plot untuk split 'validation'
plot_length_distribution(tydiqa_gold['validation'], axs[1], "Distribusi Panjang Jawaban (Validation)")

# Menampilkan plot
plt.tight_layout()  # Menyesuaikan layout agar tidak saling bertumpukan
plt.show()


In [ ]:
# Filter instance yang panjang jawabannya adalah 10 kata
instances_10_words = [row for row in tydiqa_gold['train'] if len(row['answers']['text'][0].split()) == 10]

# Tampilkan beberapa contoh
print(f"Jumlah instance dengan jawaban 10 kata: {len(instances_10_words)}\n")

for i, example in enumerate(instances_10_words[:4]):  # Ambil 5 contoh pertama
    print(f"Contoh {i+1}:")
    print(f"Pertanyaan: {example['question_text']}")
    print(f"Jawaban: {example['answers']['text'][0]}")
    print(f"Konteks: {example['passage_text'][:200]}...")  # Hanya tampilkan sebagian konteks untuk ringkas
    print("-" * 80)


Inspect per row untuk split train dan dev

In [ ]:
mr_tydi['train'][0]

In [ ]:
mr_tydi['dev'][0]